# Zusammenfassung der Zwischenergebnisse

## Festgelegte Features

Die Auswahl weiterer Features für das Vorhersagemodell basiert auf ihrer logischen Relevanz und der Analyse ihrer Korrelation mit den Verkaufszahlen.

Der Wochentag (DayOfWeek) hat oft einen erheblichen Einfluss auf die Verkäufe, da die Kaufgewohnheiten der Kunden je nach Wochentag variieren können. Beispielsweise gibt es mehr Einkäufe an Wochenenden. Daher wird der Wochentag in Liste der verwendbaren Merkmale aufgenommen.

Das Merkmal Customers (Kundenanzahl) hingegen ist weniger geeignet. Diese Entscheidung basiert auf der Abwägung der Auswirkungen auf die Modellgenauigkeit und die Komplexität des Vorhersageprozesses. Die Einbeziehung dieses Features würde bedeuten, dass auch die Anzahl der Kunden für zukünftige Zeitpunkte vorhersagt werden müsste. Dies fügt eine zusätzliche Schicht von Komplexität hinzu, da nun zwei miteinander verknüpfte Vorhersagen getroffen werden müssten: die Anzahl der Kunden und darauf basierend die Verkaufszahlen. Die Vorhersage der Kundenanzahl würde zusätzliche Modelle oder Algorithmen erfordern, die ebenfalls eine gewisse Fehlerquote aufweisen. Diese Fehler würden sich in der Verkaufsprognose widerspiegeln und könnten die Genauigkeit des Verkaufsprognosemodells verringern. Durch den Ausschluss des Customers-Features wird eine potenzielle Fehlerquelle reduziert und der Fokus liegt auf direkt beobachtbaren und prognostizierbaren Merkmalen. Der bewusste Ausschluss dieses Features hilft dabei, die Vorhersagegenauigkeit zu maximieren und die Komplexität des Modells zu minimieren, was zu einem effizienteren und zuverlässigeren Vorhersagemodell führt.

Ob ein Geschäft geöffnet oder geschlossen ist (Open), hat einen direkten Einfluss auf die Verkäufe. Geschäfte, die geschlossen sind, haben keine Verkäufe. Die Korrelationsmatrix zeigt erwartungsgemäß eine hohe Korrelation mit Sales, da Verkäufe nur an geöffneten Tagen stattfinden können.

Promotionen (Promo) beeinflussen die Verkaufszahlen erheblich, indem sie mehr Kunden anziehen und den Umsatz steigern. Die Korrelationsmatrix weist darauf hin, dass Promotionen zu höheren Verkaufszahlen führen, da eine positive Korrelation mit Sales besteht.

Feiertage (StateHoliday) beeinflussen das Einkaufsverhalten der Kunden, da an staatlichen Feiertagen Geschäfte geschlossen sein oder weniger Kunden haben könnten. Die Korrelation variiert je nach Art des Feiertags, zeigt aber signifikante Zusammenhänge.

Unterschiedliche Store-Typen (StoreType) können unterschiedliche Verkaufsmuster haben. Größere Geschäfte oder spezielle Geschäftsmodelle könnten höhere Verkäufe generieren. Die Korrelationsmatrix zeigt, dass die Korrelation je nach Typ variiert, aber signifikante Korrelationen mit anderen wichtigen Features wie Promo bestehen.

Die Wahl der zusätzlichen Features basiert auf ihrer logischen Relevanz und bisherigen Korrelationsanalysen. Features wie DayOfWeek, Open, Promo, StateHoliday und StoreType sind stark mit den Verkaufszahlen verbunden und bieten wertvolle Informationen zur Verbesserung der Modellgenauigkeit. Eine detaillierte Untersuchung der Korrelationen dieser Features mit Sales bestätigt ihre Bedeutung und rechtfertigt ihre Einbeziehung in das Vorhersagemodell.

## Änderungen

### Ausschluss von Features

Um die relevanten Features für das Modell auszuwählen, wurden verschiedene Merkmale getestet, um deren Einfluss auf die Vorhersagegenauigkeit zu bewerten. Dabei zeigte sich, dass die **Lag-Features**, die Genauigkeit des Modells erheblich verschlechterten. Aufgrund dieses negativen Effekts wurden diese Features in den weiteren Modellentwicklungen weggelassen.

### Neue Features 

Durch zuvor erwähnten Tests der unterschiedlichen Feature-Kombinationen wurde der Einfluss der einzelnen Variablen auf die Modellleistung ermittelt. Basierend auf diesen Erkenntnissen wurden die finalen Features für das Modell festgelegt, da sie die bestmögliche Leistung ohne weitere Optimierungen erreichten.

Die Zeitmerkmale Jahr, Monat, Tag und Woche des Jahres helfen dabei, saisonale Trends und zeitabhängige Muster zu erfassen. Verkäufe variieren häufig in bestimmten Monaten oder Jahreszeiten, was die Vorhersagegenauigkeit deutlich erhöht. Die Verwendung von Fourier-Transformationen, speziell die Fourier-Sin- und Fourier-Cos-Funktionen für 365 Tage, ermöglicht es, saisonale Effekte auf kontinuierliche Weise zu modellieren. Diese Features sind besonders nützlich, um komplexe saisonale Muster zu identifizieren und verbessern die Robustheit der Vorhersagen.
Die Merkmale `days_since_last_holiday` und `days_until_next_holiday` `wurden gewählt, da Feiertage oft einen signifikanten Einfluss auf die Verkaufszahlen haben. Diese zeitlichen Angaben helfen dem Modell, den Einfluss von Feiertagen auf den Verkauf in den relevanten Zeiträumen zu erfassen.

`Promo` und `Promo2` geben an, ob während bestimmter Zeiträume Werbeaktionen durchgeführt werden. Promotionen haben häufig einen erheblichen Einfluss auf die Verkaufszahlen, und die Berücksichtigung dieser Merkmale ermöglicht es dem Modell, den Effekt von Rabatten auf die Verkaufsprognosen zu lernen.

Verkaufszahlen variieren oft an verschiedenen Wochentagen, und die Codierung des Features `day_of_the_week` erlaubt es dem Modell, wöchentliche Muster zu erkennen. Das `StoreType`-Feature ist relevant, da die Art des Geschäfts einen Einfluss auf die Verkaufszahlen haben kann. Unterschiedliche Geschäftsmodelle ziehen unterschiedliche Kunden an und verfolgen variierende Verkaufsstrategien.

Ein weiteres wichtiges Feature ist `StateHoliday`, das angibt, ob ein Feiertag vorliegt. Feiertage haben oft einen starken Einfluss auf den Einzelhandel, und das Modell kann lernen, diese Effekte zu berücksichtigen. Das `Assortment`-Feature, das die Produktvielfalt im Geschäft beschreibt, ist ebenfalls entscheidend, da unterschiedliche Sortimentstypen verschiedene Kunden anziehen und die Verkaufszahlen beeinflussen können. Schließlich liefert das `Store`-Feature Informationen über spezifische Geschäfte, was hilft, lokale Verkaufsunterschiede zu erfassen. Jedes Geschäft kann unterschiedliche Verkaufsleistungen aufweisen, die durch Faktoren wie Lage, Größe und Zielgruppe bedingt sind.